# TextCNN on NLP

Traditionally, for Natural Language Processing (NLP), we used recurrent neural networks (RNN) to process the text data. In fact, we can also treat text as a one-dimensional image, so that we can use one-dimensional convolutional neural networks (CNN) to capture associations between adjacent words. This notebook describes a groundbreaking approach to applying convolutional neural networks to text analysis: textCNN [by Kim et al.](https://arxiv.org/abs/1408.5882). 

First, import the packages and modules required for the experiment.

In [2]:
import d2l
from mxnet import gluon, init, np, npx
from mxnet.contrib import text
from mxnet.gluon import nn
npx.set_np()


Text classification is a common task in NLP, which transforms a sequence of text of indefinite length into a category of text. Sentiment analysis is a popular subfield of text classification, which aims to analyze the emotions of the text’s author.

Here we use Stanford’s [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) as the dataset for sentiment analysis. The training and testing dataset each contains 25,000 movie reviews downloaded from IMDb, respectively. In addition, the number of comments labeled as “positive” and “negative” is equal in each dataset.

We are using a built-in function `load_data_imdb` for the purpose of simplicity. If you are interested in the preprocessing of the full dataset, please check [more details](https://d2l.ai/chapter_natural-language-processing/sentiment-analysis.html) at D2L.ai.

In [3]:
batch_size = 64
train_iter, test_iter, vocab = d2l.load_data_imdb(batch_size)

## One-Dimensional Convolutional Layer

Before introducing the model, let’s explain how a one-dimensional convolutional layer works. Like a two-dimensional convolutional layer, a one-dimensional convolutional layer uses a one-dimensional cross-correlation operation. In the one-dimensional cross-correlation operation, the convolution window starts from the leftmost side of the input array and slides on the input array from left to right successively. When the convolution window slides to a certain position, the input subarray in the window and kernel array are multiplied and summed by element to get the element at the corresponding location in the output array. 

As shown in figure below, the input is a one-dimensional array with a width of 7 and the width of the kernel array is 2. As we can see, the output width is  7−2+1=6  and the first element is obtained by performing multiplication by element on the leftmost input subarray with a width of 2 and kernel array and then summing the results.

![One-dimensional cross-correlation operation.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/conv1d.svg?sanitize=true)
<img src="https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/conv1d.svg?sanitize=true">

In [2]:
X = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = np.array([[0, 1], [2, 3]])
corr2d(X, K)

array([[19., 25.],
       [37., 43.]])

Convolutional layers

In [3]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1,))

    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

Padding

In [4]:
# A convenient function to test Gluon convoplution layers. 
def comp_conv2d(conv2d, X):
    conv2d.initialize()
    # Add batch and channel dimension.
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Exclude the first two dimensions
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2D(1, kernel_size=3, padding=1)
X = np.random.uniform(size=(8, 8))
comp_conv2d(conv2d, X).shape

(8, 8)

Stride

In [5]:
conv2d = nn.Conv2D(1, kernel_size=3, padding=1, strides=2)
comp_conv2d(conv2d, X).shape

(4, 4)

A slightly more complicated example

In [6]:
conv2d = nn.Conv2D(1, kernel_size=(3, 5), padding=(0, 1), strides=(3, 4))
comp_conv2d(conv2d, X).shape

(2, 2)

Multiple input channels

In [7]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K))

X = np.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = np.array([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)

array([[ 56.,  72.],
       [104., 120.]])

Multiple output channels

In [8]:
def corr2d_multi_in_out(X, K):
    return np.stack([corr2d_multi_in(X, k) for k in K])

K = np.stack((K, K + 1, K + 2))
K.shape, corr2d_multi_in_out(X, K).shape

((3, 2, 2, 2), (3, 2, 2))